In [1]:
import os
import joblib
from model_settings import ms
from pathlib import Path
ms.find_root(Path())
models_dir = os.path.join(ms.root,ms.trained_models)
models = [f for f in os.listdir(models_dir) if f.find('.')==-1]
for i,m in enumerate(models):
    print(f"{i}  {m}")
i = int(input('select model: '))
model_dir = os.path.join(models_dir,models[i])
pickle = os.path.join(model_dir,models[i]+'.pkl')

0  2024-12-03 115353819604 inital cboe spx relative asian
1  2024-12-03 221708375538 inital cboe spx relative barrier


select model:  0


In [2]:
model = joblib.load(pickle)

In [5]:
model['mlp_params']

{'solver': 'sgd',
 'activation': 'relu',
 'max_iter': 500,
 'random_state': 1312,
 'early_stopping': True,
 'hidden_layer_sizes': (20,),
 'learning_rate': 'adaptive',
 'learning_rate_init': 0.1,
 'n_iter_no_change': 20,
 'tol': 0.0001,
 'warm_start': False}

In [6]:
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
model['mlp_params']['verbose']=True
mlp = MLPRegressor(**model['mlp_params'])
mlp.out_activation_= 'relu'
regressor = Pipeline([
            ("preprocessor", model['preprocessor']),
            ("regressor", mlp)
        ])

In [7]:
regressor.fit(model['train_X'],model['train_y'])

Iteration 1, loss = 0.02173819
Validation score: 0.960354
Iteration 2, loss = 0.00121118
Validation score: 0.976049
Iteration 3, loss = 0.00090158
Validation score: 0.978766
Iteration 4, loss = 0.00053727
Validation score: 0.993335
Iteration 5, loss = 0.00014939
Validation score: 0.998274
Iteration 6, loss = 0.00005177
Validation score: 0.999166
Iteration 7, loss = 0.00003190
Validation score: 0.999414
Iteration 8, loss = 0.00002570
Validation score: 0.999506
Iteration 9, loss = 0.00002296
Validation score: 0.999554
Iteration 10, loss = 0.00002142
Validation score: 0.999585
Iteration 11, loss = 0.00002052
Validation score: 0.999596
Iteration 12, loss = 0.00001988
Validation score: 0.999617
Iteration 13, loss = 0.00001949
Validation score: 0.999635
Iteration 14, loss = 0.00001913
Validation score: 0.999650
Iteration 15, loss = 0.00001886
Validation score: 0.999652
Iteration 16, loss = 0.00001861
Validation score: 0.999656
Iteration 17, loss = 0.00001850
Validation score: 0.999657
Iterat

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  ['days_to_maturity',
                                                   'fixing_frequency',
                                                   'past_fixings',
                                                   'risk_free_rate',
                                                   'dividend_rate', 'kappa',
                                                   'theta', 'rho', 'eta', 'v0',
                                                   'relative_spot']),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['averaging_type', 'w'])])),
                ('regressor',
                 MLPRegressor(early_stopping=True, hidden_layer_sizes=(20,),
                              learning_rate='adaptive', learning_rate_init=0.1,
                              max_iter=500, n_iter_no_change=20,
                              random_state=1312, solver='sgd', verbose=True))])

In [18]:
import numpy as np
inpred = regressor.predict(model['train_X'])
intar = model['train_data']['insample_prediction'].values
indiff = inpred-intar
inRMSE = np.sqrt(np.mean(indiff**2))
inMAE = np.mean(np.abs(indiff))
print(f"insample\n   RMSE:\n      {inRMSE}")
print(f"insample\n   MAE:\n      {inMAE}")

insample
   RMSE:
      0.00015370595125211
insample
   MAE:
      4.834968647423211e-05


In [19]:
outpred = regressor.predict(model['test_X'])
outtar = model['test_data']['outofsample_prediction'].values
outdiff = outpred-outtar
outRMSE = np.sqrt(np.mean(outdiff**2))
outMAE = np.mean(np.abs(outdiff))
print(f"out of sample\n   RMSE:\n      {outRMSE}")
print(f"out of sample\n   MAE:\n      {outMAE}")

out of sample
   RMSE:
      0.0353261435466088
out of sample
   MAE:
      0.01641081387073828
